<a href="https://colab.research.google.com/github/Ben93kie/SeaDronesSee/blob/main/MOT/visualize_bbox_on_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualize BoundingBoxes on a video

The script will take images and the annotations in COCO-Format to visualize the bounding boxes on these images and create a video

Lets say, we have the images below and the corresponding annotation given and want to visualize the Bounding Boxes on this images and output a video



## Get the images and the annotations
first run the code to download the images

In [20]:
import requests
import zipfile

url = "https://cloud.cs.uni-tuebingen.de/index.php/s/PdKfdJFAFmLYWAz/download"
r = requests.get(url)
with open("/content/sample_data/imgs", 'wb') as f:
  f.write(r.content)

with zipfile.ZipFile("sample_data/imgs", 'r') as myzip:
    myzip.extractall()

now download the annotations

In [21]:
import requests

url = "https://cloud.cs.uni-tuebingen.de/index.php/s/pNqRDqtLjXbEqDm/download?path=%2Fannotations&files=instances_val.json&downloadStartSecret=o3mqezb18tr"
r = requests.get(url)
with open("/content/sample_data/annotation.json", 'wb') as f:
  f.write(r.content)

## Available Images
are the images are saved in /content/Sample_images

## Import required packages

first we import all the needed packages

In [22]:
import argparse
import json

import cv2
import time
import os

## Color assignment
for better visualization of each category we assign each category a different color (excluding category nr 7 - ignored)
1. <font color='00FF00'>swimmer</font>
2. <font color='FF0000'>floater</font>
3. <font color='33FFFF'>boat</font>
4. <font color='FFFF00'>swimmer on boat</font>
5. <font color='FF66FF'>floater on boat</font>
6. <font color='FF9933'>life jacket</font>
7. ignored

***Attention: The color values are not RGB (Red, Green, Blue) but BGR (Blue, Green, Red)***


In [23]:
colors = {1: (0,255,0),
          2: (0,0,255),
          3: (255,255,51),
          4: (0,255,255),
          5: (255, 102, 255),
          6: (51, 153, 255)}

## Reading the annotation file
we open the annotation json file, which is a dictonary

In [24]:
with open("sample_data/annotation.json") as f:
  data = json.load(f)

## Video Settings

the width and height of the video will be determined automatically (size of the input image)

The used Videocodex will produce an .avi video

The fps (Frames per Second) is set to 2 for demonstration purposes

In [30]:
width = data['images'][0]['width']
height = data['images'][0]['height']
shape = width, height
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
fps = 2
out = cv2.VideoWriter("/content/output.avi", fourcc, fps, shape)

## Drawing the Bounding Boxes
we go through all the images and draw the bounding boxes, which are defined in the annotation file, on the images.

In the annotation file the bounding boxes are saved as an array with length 4.

| index | meaning|
| --- | --- |
| 0 | x coordinate of the top left corner of the bounding box
| 1 | y coordinate of the top left corner of the bounding box
| 2 | width of the bounding box
| 3 | height of the bounding box

with these information we can then calculate the lower right point of the bounding box by adding width to the x coordinate and adding the height to the y coordinate. This is needed because cv2.rectangle() is using the upper left corner and lower right corner to define the rectangle to draw.

With the category_id the color for the Bounding Box is determined.

After all Bounding Boxes, which belong to one image, are drawn on the image the image will be added to the output video.

After all the images are processed, the video will be saved in the workspace

In [31]:
for image in data['images']:
  if not os.path.isfile("/content/Sample_images/" + image['file_name']):
    continue
  img = cv2.imread("/content/Sample_images/" + image['file_name'])
  for annotation in data['annotations']:
    if annotation['image_id'] == image['id']:
      topLeftX = annotation['bbox'][0]
      topLeftY = annotation['bbox'][1]
      width = annotation['bbox'][2]
      height = annotation['bbox'][3]
      cv2.rectangle(img, (topLeftX, topLeftY), (topLeftX + width, topLeftY + height), colors[annotation['category_id']], 3)
  img = cv2.resize(img, shape)
  out.write(img)
out.release()

The video is now saved in your workspace /content/output.avi